분석을 위해 필요한 라이브러리들을 불러옵니다.

1. json: JSON 데이터를 다루기 위해 사용됩니다.
2. warnings: 경고 메시지를 무시하고 싶을 때 사용됩니다.
3. numpy: 행렬 등의 수치 연산을 빠르게 수행하기 위해 사용되는 선형대수 라이브러리입니다.
4. tqdm: 모델의 학습 과정을 시각화하기 위해 사용됩니다.
5. sklearn.metrics: 모델 성능 평가의 다양한 metric을 scikit-learn 라이브러리에서 가져옵니다.
    1. precision_score: 정밀도
    2. recall_score: 재현율
    3. accuracy_score: 정확도
    4. f1_score: 정밀도와 재현율의 조화평균


분석을 위해 미리 정의해 둔 파이썬 파일들을 불러옵니다.
- module 디렉터리 내부에 존재하는 여러 파이썬 파일들을 가져옵니다.

1. argument: 파일을 실행할 때, 추가적으로 argument를 받을 수 있도록 하는 함수를 포함합니다.
    1. get_parser: 추가적인 argument로 어떤 변수들을 받을 지 설정합니다.(자세한 내용은 argument.py에서 확인할 수 있음)

2. read_data: 데이터를 불러오는 함수를 포함하고 있는 파이썬 파일입니다.
    1. load_data: 데이터를 불러오는 함수입니다.

3. smiles2fing: SMILES 표기법을 fingerprint로 변환하기 위한 함수를 포함하고 있는 파이썬 파일입니다.
4. get_model: 모델을 불러오는 함수를 포함하고 있는 파이썬 파일입니다.
    1. load_model: 모델을 불러오는 함수입니다.
    2. load_hyperparameter: 모델의 하이퍼파라미터를 불러오는 함수입니다.

5. common: 분석을 위해 필요한 미리 정의해 둔 여러 잡다한 함수들을 포함하고 있는 파이썬 파일입니다.
    1. data_split: 데이터를 train, validation, test 데이터로 분리하는 함수입니다.
    2. multiclass_cross_validation: 다중 분류 문제를 위한 교차 검증을 수행하는 함수입니다.
    3. print_best_param: 교차 검증을 수행한 결과를 출력하는 함수입니다.

In [ ]:
import json
import warnings

import numpy as np
from tqdm import tqdm

from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    f1_score
)

from module.argument import get_parser
from module.read_data import load_data
from module.smiles2fing import smiles2fing
from module.get_model import (
    load_model,
    load_hyperparameter
)
from module.common import (
    data_split, 
    multiclass_cross_validation,
    print_best_param
)


warnings.filterwarnings('ignore')

아래에 이어지는 셀들은 모든 main함수를 구성합니다.
- 실제로 함수를 동작사키기 위해서는 분할된 셀을 모두 합쳐놓은 상태에서 실행해야 합니다.


argparse.ArgumentParser()의 인스턴스로 parser를 정의합니다.


try-except문은 예외처리를 위한 구문입니다. try문 안에 실행할 코드를 넣고, except문 안에 예외 발생 시 실행할 코드를 넣습니다. 예외가 발생하지 않으면 except문은 실행되지 않습니다.

parse_args()는 커맨드 라인(터미널)에서 제공된 인자들을 파싱해, 각 인자들을 변수로 변환합니다.
- 사용자가 정의되어있지 않은 인자를 제공하거나, 커맨드 라인 인터페이스의 적절한 규칙을 따르지 않은 경우, parse_args()의 호출이 실패할 수 있습니다.
- 이 경우, except에서 미리 지정해 둔 대로, 빈 리스트에 대해 파싱을 수행합니다.
- 따라서 아무런 인자도 넣어주지 않은 상태와 동일하게, 모든 argument가 default 값으로 프로그램이 실행됩니다.

In [ ]:
def main():
    parser = get_parser()
    try:
        args = parser.parse_args()
    except:
        args = parser.parse_args([])
    print('=================================')

선택한 데이터 파일의 tg_num(403, 412, 413), inhale_type(aerosol, vapour), 그리고 분석을 수행할 모델(logistic, dt, ...)의 이름을 출력합니다.

load_data 함수(read_data 항목 참조)를 이용해 fingerprint로 변환한 데이터프레임을 x에, 타겟을 y에 저장합니다.

변수 x, y에 저장해 둔 데이터를 data_split 함수(common 항목 참조)를 통해 학습 데이터와 트레인 데이터로 분할합니다.
- 커맨드 라인에서 제공된(제공하지 않을 경우, default 값으로 설정됨) splitseed에 따라 분할됩니다.

In [ ]:
    print('Multiclass tg%s %s %s' % (args.tg_num, args.inhale_type, args.model))
    
    x, y = load_data(path = 'data', tg_num = args.tg_num, inhale_type = args.inhale_type)
    
    x_train, x_test, y_train, y_test = data_split(x, y, args.splitseed)

선택한 모델의 하이퍼파라미터를 load_hyperparameter(get_model 항목 참조) 함수를 통해 가져와, params에 저장합니다.
- params에는 실험자가 설정한 하이퍼파라미터의 범위 내의 가능한 모든 조합이 저장됩니다.
- 따라서, 아래의 for문에서 가능한 모든 조합에 대한 성능을 측정하게 됩니다.

결과를 저장하기 위한 dictionary, result를 생성합니다.
result의 key로 model, precision, recall, f1, accuracy를 생성합니다.


가능한 모든 하이퍼파라미터의 조합에 대한 성능을 각각 평가합니다.
result에 몇 번째 하이퍼파라미터 조합에 대한 성능인지 확인하기 위해 인덱스를 붙혀 저장합니다.


특정 조합에서 커맨드 라인에서 설정한 반복 횟수(args.num_run)만큼 동일한 과정을 반복합니다.
- 동일 과정을 반복하는 이유는, 모델의 성능이 데이터에 따라 달라지기 때문입니다.
    - 학습 데이터셋의 분할(학습 데이터와 검증 데이터로 분할)은 multiclass_cross_validation 함수(common 항목 참조)를 통해 수행됩니다.
    - validation set을 통해 모델의 성능을 평가할 때, 데이터의 분할이 달라질 수 있습니다.
        - 데이터 분할은 seed를 통해 결정됩니다.

    - 따라서, 데이터의 분할에 따른 모델의 성능의 변화를 확인하기 위해 반복합니다.

- 반복 횟수만큼 반복하며, 각각의 반복에서의 성능을 평가하고 이를 result에 저장합니다.



json.dump는 dictionary를 json 파일로 저장하는 함수입니다.
- 실험 결과를 저장하기 위해, result를 json 파일로 저장합니다.

print_best_param 함수(common 항목 참조)는 validation set을 통해 측정한 성능을 기준으로 평가할 때, 가장 좋은 성능을 보인 하이퍼파라미터 조합을 반환하는 함수입니다.

변수 m에 가장 좋은 성능을 보인 하이퍼파라미터 조합의 인덱스('model')를 저장합니다.
- 예시) m = 'model0'

In [ ]:
    # cross validation
    params = load_hyperparameter(args.model)

    result = {}
    result['model'] = {}
    result['precision'] = {}
    result['recall'] = {}
    result['f1'] = {}
    result['accuracy'] = {}

    for p in tqdm(range(len(params))):
        
        result['model']['model'+str(p)] = params[p]
        result['precision']['model'+str(p)] = []
        result['recall']['model'+str(p)] = []
        result['f1']['model'+str(p)] = []
        result['accuracy']['model'+str(p)] = []
        
        for seed in range(args.num_run):
            model = load_model(model = args.model, seed = seed, param = params[p])
            
            cv_result = multiclass_cross_validation(model, x_train, y_train, seed)
            
            result['precision']['model'+str(p)].append(cv_result['val_precision'])
            result['recall']['model'+str(p)].append(cv_result['val_recall'])
            result['f1']['model'+str(p)].append(cv_result['val_f1'])
            result['accuracy']['model'+str(p)].append(cv_result['val_accuracy'])

    json.dump(result, open(f'tg{args.tg_num}_val_results/multiclass/{args.inhale_type}_{args.model}.json', 'w'))
    
    
    best_param = print_best_param(val_result = result, metric = args.metric)
    m = list(result['model'].keys())[list(result['model'].values()).index(best_param)]


m에 해당되는 모델의 validation 결과를 각 metric 별로 변수에 저장하고 이를 출력합니다.
- 하이퍼파라미터가 어떻게 설정되어있지도 출력합니다.
- 각 모델마다 10번(default값, argument 항목 참조)의 cross validation을 수행하였으므로, metric의 평균과 표준편차를 출력합니다.

In [ ]:
    # val result
    precision = result['precision'][m]
    recall = result['recall'][m]
    acc = result['accuracy'][m]
    f1 = result['f1'][m]
    
    print(f"best param: {best_param} \
          \nvalidation result \
          \nprecision: {np.mean(precision):.3f}({np.std(precision):.3f}) \
          \nrecall: {np.mean(recall):.3f}({np.std(recall):.3f}) \
          \naccuracy: {np.mean(acc):.3f}({np.std(acc):.3f}) \
          \nf1: {np.mean(f1):.3f}({np.std(f1):.3f})")

validation 결과상, 성능이 가장 좋았던 하이퍼파라미터를 따르는 모델을 테스트셋에 적용하여 성능을 확인합니다.
학습 데이터셋에 대해 학습을 시킨 후, 테스트셋에 대한 성능을 측정합니다.


plsda 모델의 경우, 다중분류를 위해 one-hot encoding을 적용합니다.
- 이를 위해 학습 데이터셋에 대해 one-hot encoding을 적용합니다.
- 또한 테스트셋에 대한 예측값에 대해 argmax를 적용하여 다중분류를 위한 예측값으로 변환합니다.


각 평가지표에 대한 성능을 출력합니다.

In [ ]:
    # test reulst
    model = load_model(model = args.model, seed = seed, param = best_param)
    
    if args.model == 'plsda':
        y_train = pd.get_dummies(y_train)
    
    model.fit(x_train, y_train)
    pred = model.predict(x_test)
    
    if args.model == 'plsda':
        pred = np.argmax(pred, axis = 1)
    
    print(f'test result \
          \nbest param: {best_param} \
          \nprecision: {precision_score(y_test, pred, average = "macro"):.3f} \
          \nrecall: {recall_score(y_test, pred, average = "macro"):.3f} \
          \naccuracy: {accuracy_score(y_test, pred):.3f} \
          \nf1: {f1_score(y_test, pred, average = "macro"):.3f}')

main함수를 실행시켜, 위의 모든 과정을 수행합니다.

In [ ]:
if __name__ == '__main__':
    main()